In [1]:
import pandas as pd
import numpy as np
from datetime import date
from statistics import mean, mode
from pandas.api.types import CategoricalDtype
import math
from scipy.stats import norm

In [2]:
today = date.today()
print(today)

2024-01-07


# Tables

In [3]:
path_tables = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 2 tables/"

In [4]:
# tables
filename_careerorglink = "careerorglink.xlsx"
filename_leadercareerlink = "leadercareerlink.xlsx"
filename_orgtree = "orgtree.xlsx"
filename_elected = "positions_elected.xlsx"

In [5]:
# career-org link
# col = pd.read_excel(path_tables + filename_careerorglink,dtype="str")
# col.shape

In [6]:
# col.columns

In [7]:
# leader-career link
# lcl = pd.read_excel(path_tables + filename_leadercareerlink,dtype="str")
# lcl.shape

In [8]:
# lcl.columns

In [9]:
# orgtree
# org = pd.read_excel(path_tables + filename_orgtree,dtype="str")
# org.shape

In [10]:
# org.columns

In [11]:
# elected = pd.read_excel(path_tables + filename_elected,dtype="str")
# elected.shape

In [12]:
# elected.columns

# Queries

In [13]:
path_queries = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 3 queries/"

In [14]:
filename_leaderjob_all = "leaderjob_electUnelect_inOutgov.xlsx"
filename_leaderjob_no_spa = "leaderjob_no_spa.xlsx"
filename_leaderjobtransition_no_spa = "leaderjobtransition_no_spa.xlsx"

In [15]:
# leader jobs
# ljobs_all = pd.read_excel(path_queries + filename_leaderjob_all,dtype="str")
# ljobs_all.shape

In [16]:
# ljobs = pd.read_excel(path_queries + filename_leaderjob_no_spa,dtype="str")
# ljobs.shape

In [17]:
# transitions used for analysis - no SPA - no local-local
trans = pd.read_excel(path_queries + filename_leaderjobtransition_no_spa,dtype="str")
trans.shape

(4306, 31)

In [20]:
trans.columns

Index(['LeaderID', 'CareerString_1', 'CareerDateString_2022_1',
       'CareerStartYear_1', 'CareerStartMonth_1', 'CareerStartDate_1',
       'CareerSubstring_1', 'InstitutionType_1', 'PrimaryInstitution_1',
       'OrgName_1', 'Local_1', 'Position_1', 'IsElected_1', 'OrgRank_1',
       'PositionRank_1', 'CareerString_2', 'CareerDateString_2022_2',
       'CareerStartYear_2', 'CareerStartMonth_2', 'CareerStartDate_2',
       'CareerSubstring_2', 'InstitutionType_2', 'PrimaryInstitution_2',
       'OrgName_2', 'Local_2', 'Position_2', 'IsElected_2', 'OrgRank_2',
       'PositionRank_2', 'OrgAdvance', 'PositionAdvance'],
      dtype='object')

# Analysis - Research Note

In [18]:
path_analysis = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/analysis/"

In [19]:
# analysis sub-paths
study0_path = "2023.10.04 Study 0 - research note/"
study1_path = "2023.10.04 Study 1 - political capital/"
study2_path = "2023.10.04 Study 2 - commitment vs control/"
study3_path = "2023.10.04 Study 3 - reds vs experts/"

In [121]:
# ljobs = ljobs.astype({"CareerStartYear":"int","CareerStartDate":"int"})
# ljobs.dtypes

# Functions

In [122]:
def merge_results(m):
    
    print("\nMerge Results...")
    print("")
    print("\tshape     :",m.shape)
    print("\tleft_only :",m[m["_merge"]=="left_only"].shape)
    print("\tboth      :",m[m["_merge"]=="both"].shape)
    print("\tright_only:",m[m["_merge"]=="right_only"].shape)

In [123]:
# using this on (PI,OrgName) will ensure unique & non-null keys
# using this on a larger df will ensure unique rows and non-null keys, but not unique keys

def unique_non_null_rows(olddf):
    
    df = olddf.copy()
    
    
    ### drop duplicates
    df = df.drop_duplicates(keep="first",ignore_index=True)

    ### drop null rows
    df.dropna(how="all",axis=0,inplace=True)
    
    # drop rows with null PI
    df = df[~(df["PrimaryInstitution"].isna())]
        
    ### drop rows where PI contains stop words
    stop_words_lower = ["uncertain","current","deprecated","please_revise"]
    df = df[ ~ df["PrimaryInstitution"].str.lower().isin(stop_words_lower)]
            

    print("\nUnique Non-null Rows...")
    print("")
    print("\tNon-unique rows:",olddf.shape)
    print("\tUnique rows    :",df.shape)

    ### sort
    df = df.sort_values(["PrimaryInstitution","OrgName"])
    
    return df

In [124]:
def create_time_series(series,group_var,count_var):
    
    yeardist = series.groupby(group_var,as_index=False).count().sort_values(group_var)
    
    max_year = max(series[group_var])
    x = pd.DataFrame({"year":pd.Series(range(min(yeardist[group_var].astype(int)),max(yeardist[group_var].astype(int))+1))})
    
    yeardist[group_var] = yeardist[group_var].astype(int,errors="ignore")
    
    ts = x.merge(yeardist,left_on="year",right_on=group_var,how="left")
    ts.loc[ts[count_var].isna(),count_var]=0
    # merge_results(ts)
    
    ts_columns = ["year",count_var]
    ts = ts[ts_columns]
    
    return ts

In [125]:
# create InstitutionCategory (English) variable InstitutionType

# use with apply. e.g., 
# df["InstitutionCategory] = df["InstitutionType"].apply(define_institution_category)

def define_institution_category(PI):
    
    c = "Social"
    
    if PI=="정권기관":
        c = "Government"
    elif PI=="노동당":
        c = "Party"
    elif PI=="인민군":
        c = "Military"
        
    return c

In [126]:
def two_sample_ttest_proportions(x1,x2,n1,n2,alpha):
    
    pstar = (x1+x2)/(n1+n2)
    p1 = x1/n1
    p2 = x2/n2
    z = (p1-p2) / math.sqrt(pstar*(1-pstar)*((1/n1) + (1/n2)))
    #pvalue = 
    print("p1:\t",p1)
    print("p2:\t",p2)
    print("p1-p2:\t",(p1-p2))
    print("z:\t",z)
    print("pvalue:\t","calculate")

# Format & Covariates

### 0. change datatypes

In [127]:
trans = trans.astype({"OrgAdvance":"int","PositionAdvance":"int",
                      "CareerStartYear_1":"int","CareerStartYear_2":"int",
                      "CareerStartDate_1":"int","CareerStartDate_2":"int"})
trans.dtypes

LeaderID                   object
CareerString_1             object
CareerDateString_2022_1    object
CareerStartYear_1           int32
CareerStartMonth_1         object
CareerStartDate_1           int32
CareerSubstring_1          object
InstitutionType_1          object
PrimaryInstitution_1       object
OrgName_1                  object
Local_1                    object
Position_1                 object
IsElected_1                object
OrgRank_1                  object
PositionRank_1             object
CareerString_2             object
CareerDateString_2022_2    object
CareerStartYear_2           int32
CareerStartMonth_2         object
CareerStartDate_2           int32
CareerSubstring_2          object
InstitutionType_2          object
PrimaryInstitution_2       object
OrgName_2                  object
Local_2                    object
Position_2                 object
IsElected_2                object
OrgRank_2                  object
PositionRank_2             object
OrgAdvance    

### 1. double-check exlcudes SPA jobs and local-local trans

In [128]:
trans.columns

Index(['LeaderID', 'CareerString_1', 'CareerDateString_2022_1',
       'CareerStartYear_1', 'CareerStartMonth_1', 'CareerStartDate_1',
       'CareerSubstring_1', 'InstitutionType_1', 'PrimaryInstitution_1',
       'OrgName_1', 'Local_1', 'Position_1', 'IsElected_1', 'OrgRank_1',
       'PositionRank_1', 'CareerString_2', 'CareerDateString_2022_2',
       'CareerStartYear_2', 'CareerStartMonth_2', 'CareerStartDate_2',
       'CareerSubstring_2', 'InstitutionType_2', 'PrimaryInstitution_2',
       'OrgName_2', 'Local_2', 'Position_2', 'IsElected_2', 'OrgRank_2',
       'PositionRank_2', 'OrgAdvance', 'PositionAdvance'],
      dtype='object')

In [129]:
trans.shape

(4306, 31)

In [130]:
trans.Local_1.unique()

array(['False', 'True'], dtype=object)

In [131]:
# remove Local-Local ties
trans = trans[~((trans["Local_1"]=="True") & (trans["Local_2"]=="True"))]
trans.shape

(4222, 31)

### 2. add InstitutionCategory_1, InstitutionCategory_2

In [132]:
trans.InstitutionType_1.unique()

array(['정권기관', '국제친선단체', '노동당', '인민군', '당외곽및사회단체_사회부문(별책)',
       '당외곽및사회단체_사회부문', '당외곽및사회단체_체육부문', '당외곽및사회단체_정치부문', '당외곽및사회단체_대외부문',
       '당외곽및사회단체_근로단체', '당외곽및사회단체_종교부문', '당외곽및사회단체_경제부문(별책)',
       '당외곽및사회단체_경제부문'], dtype=object)

### 3. PISame, OrgSame

In [133]:
trans.columns

Index(['LeaderID', 'CareerString_1', 'CareerDateString_2022_1',
       'CareerStartYear_1', 'CareerStartMonth_1', 'CareerStartDate_1',
       'CareerSubstring_1', 'InstitutionType_1', 'PrimaryInstitution_1',
       'OrgName_1', 'Local_1', 'Position_1', 'IsElected_1', 'OrgRank_1',
       'PositionRank_1', 'CareerString_2', 'CareerDateString_2022_2',
       'CareerStartYear_2', 'CareerStartMonth_2', 'CareerStartDate_2',
       'CareerSubstring_2', 'InstitutionType_2', 'PrimaryInstitution_2',
       'OrgName_2', 'Local_2', 'Position_2', 'IsElected_2', 'OrgRank_2',
       'PositionRank_2', 'OrgAdvance', 'PositionAdvance'],
      dtype='object')

In [134]:
trans["PISame"] = False
trans.loc[trans["PrimaryInstitution_1"]==trans["PrimaryInstitution_2"],["PISame"]] = True
trans.PISame.unique()

array([False,  True])

In [135]:
trans["OrgSame"] = np.nan
trans.loc[trans["PISame"]==True,"OrgSame"] = False
trans.loc[(trans["PISame"]==True) & (trans["OrgName_1"]==trans["OrgName_2"]),["OrgSame"]] = True
trans.OrgSame.unique()

array([nan, True, False], dtype=object)

### 4. OrgRankChange, PositionRankChange

In [136]:
trans["OrgRankChange"] = np.nan

In [137]:
trans.loc[trans["PISame"] & (trans["OrgRank_1"]>trans["OrgRank_2"]),"OrgRankChange"] = "lower"
trans.loc[trans["PISame"] & (trans["OrgRank_1"]==trans["OrgRank_2"]),"OrgRankChange"] = "same"
trans.loc[trans["PISame"] & (trans["OrgRank_1"]<trans["OrgRank_2"]),"OrgRankChange"] = "higher"

In [138]:
value_order = ["lower","same","higher"]
trans.OrgRankChange = trans.OrgRankChange.astype("category")
trans.OrgRankChange = trans.OrgRankChange.cat.set_categories(value_order)

In [139]:
trans.OrgRankChange.unique()

[NaN, 'same', 'higher', 'lower']
Categories (3, object): ['lower', 'same', 'higher']

In [140]:
trans["PositionRankChange"] = np.nan

In [141]:
trans.loc[trans["OrgSame"] & (trans["PositionRank_1"]>trans["PositionRank_2"]),"PositionRankChange"] = "lower"
trans.loc[trans["OrgSame"] & (trans["PositionRank_1"]==trans["PositionRank_2"]),"PositionRankChange"] = "same"
trans.loc[trans["OrgSame"] & (trans["PositionRank_1"]<trans["PositionRank_2"]),"PositionRankChange"] = "higher"

In [142]:
value_order = ["lower","same","higher"]
trans.PositionRankChange = trans.PositionRankChange.astype("category")
trans.PositionRankChange = trans.PositionRankChange.cat.set_categories(value_order)

In [143]:
trans.PositionRankChange.unique()

[NaN, 'same', 'lower', 'higher']
Categories (3, object): ['lower', 'same', 'higher']

### 5. Succession Period - Broad

* KIS-->KJI: 1974-1993 - less institutionalized (stronger intra)
* KJI--->KJU:  2002-2011 - more institutionalized (stronger inter)

In [144]:
trans["Succession_Broad"] = np.nan

In [145]:
value_order = ["KIS to KJI","KJI to KJU"]
trans.Succession_Broad = trans.Succession_Broad.astype("category")
trans.Succession_Broad = trans.Succession_Broad.cat.set_categories(value_order)

In [146]:
trans.loc[(trans["CareerStartYear_2"]>1973) & (trans["CareerStartYear_2"]<1994),"Succession_Broad"] = "KIS to KJI"
trans.loc[(trans["CareerStartYear_2"]>2001) & (trans["CareerStartYear_2"]<2012),"Succession_Broad"] = "KJI to KJU"

In [147]:
trans[["Succession_Broad","OrgName_2"]].groupby("Succession_Broad",as_index=False).count()

,Succession_Broad,OrgName_2
0,KIS to KJI,674
1,KJI to KJU,648


In [148]:
trans.loc[trans["Succession_Broad"]=="KIS to KJI","CareerStartYear_2"]

34      1977
35      1977
36      1979
37      1980
38      1982
        ... 
3805    1980
3934    1993
4285    1977
4286    1980
4287    1990
Name: CareerStartYear_2, Length: 1006, dtype: int32

### 6. Succession Period - Narrow

#### Esther's periodization
* KIS-->KJI: 1987-1994 - less institutionalized (stronger intra)
* KJI--->KJU:  2009-2011 - more institutionalized (stronger inter)

In [149]:
trans["Succession_Narrow"] = np.nan

In [150]:
value_order = ["KIS to KJI","KJI to KJU"]
trans.Succession_Narrow = trans.Succession_Narrow.astype("category")
trans.Succession_Narrow = trans.Succession_Narrow.cat.set_categories(value_order)

In [151]:
### Esther's periodization

trans.loc[(trans["CareerStartYear_2"]>1986) & (trans["CareerStartYear_2"]<1995),"Succession_Narrow"] = "KIS to KJI"
trans.loc[(trans["CareerStartYear_2"]>2008) & (trans["CareerStartYear_2"]<2012),"Succession_Narrow"] = "KJI to KJU"

In [152]:
trans[["Succession_Narrow","OrgName_2"]].groupby("Succession_Narrow",as_index=False).count()

,Succession_Narrow,OrgName_2
0,KIS to KJI,323
1,KJI to KJU,383


### 7. OrgAdvanceYes

In [153]:
trans["OrgAdvanceYes"] = np.nan

In [154]:
trans.loc[trans["PISame"]==True,"OrgAdvanceYes"] = False
trans.loc[trans["PISame"]==True & (trans["OrgRank_1"]>trans["OrgRank_2"]),"OrgAdvanceYes"] = True

### 8. PositionAdvanceYes

In [155]:
trans["PositionAdvanceYes"] = np.nan

In [156]:
trans.loc[trans["OrgSame"]==True,"PositionAdvanceYes"] = False
trans.loc[trans["OrgSame"]==True & (trans["PositionRank_1"]>trans["PositionRank_2"]),"PositionAdvanceYes"] = True

### 9. InstitutionCategory

In [157]:
trans.columns

Index(['LeaderID', 'CareerString_1', 'CareerDateString_2022_1',
       'CareerStartYear_1', 'CareerStartMonth_1', 'CareerStartDate_1',
       'CareerSubstring_1', 'InstitutionType_1', 'PrimaryInstitution_1',
       'OrgName_1', 'Local_1', 'Position_1', 'IsElected_1', 'OrgRank_1',
       'PositionRank_1', 'CareerString_2', 'CareerDateString_2022_2',
       'CareerStartYear_2', 'CareerStartMonth_2', 'CareerStartDate_2',
       'CareerSubstring_2', 'InstitutionType_2', 'PrimaryInstitution_2',
       'OrgName_2', 'Local_2', 'Position_2', 'IsElected_2', 'OrgRank_2',
       'PositionRank_2', 'OrgAdvance', 'PositionAdvance', 'PISame', 'OrgSame',
       'OrgRankChange', 'PositionRankChange', 'Succession_Broad',
       'Succession_Narrow', 'OrgAdvanceYes', 'PositionAdvanceYes'],
      dtype='object')

In [158]:
trans["InstitutionCategory_1"] = trans["InstitutionType_1"].apply(define_institution_category)
trans["InstitutionCategory_2"] = trans["InstitutionType_2"].apply(define_institution_category)

### 10. InstitutionCategorySame

In [159]:
trans["InstitutionCategorySame"] = trans["InstitutionCategory_1"] == trans["InstitutionCategory_2"]

### 11. Succession_Five

#### Jacob's periodization
* KIS-->KJI: 1994-1999 - less institutionalized (stronger intra)
* KJI--->KJU:  2011-2016 - more institutionalized (stronger inter)

In [160]:
trans["Succession_Five"] = np.nan

In [161]:
value_order = ["KIS to KJI","KJI to KJU"]
trans.Succession_Five = trans.Succession_Five.astype("category")
trans.Succession_Five = trans.Succession_Five.cat.set_categories(value_order)

In [162]:
### Jacob's periodization

trans.loc[(trans["CareerStartYear_2"]>1993) & (trans["CareerStartYear_2"]<2000),"Succession_Five"] = "KIS to KJI"
trans.loc[(trans["CareerStartYear_2"]>2010) & (trans["CareerStartYear_2"]<2016),"Succession_Five"] = "KJI to KJU"

In [163]:
trans[["Succession_Five","OrgName_2"]].groupby("Succession_Five",as_index=False).count()

,Succession_Five,OrgName_2
0,KIS to KJI,229
1,KJI to KJU,229


# Statistics

### 1. OrgRank increase - percent by year

In [164]:
stat1_columns = ["CareerStartYear_2","OrgAdvanceYes"]
stat1_groupby_columns = ["CareerStartYear_2"]
stat1_label_columns = ["Year","Total Transitions","Total Advancements of OrgRank"]

In [165]:
stat1 = trans[stat1_columns].groupby(stat1_groupby_columns,as_index=False)["OrgAdvanceYes"].agg({"Total Transitions":"count","Total Advancements of OrgRank":"sum"})
stat1.columns = stat1_label_columns
stat1["Percentage Advancements of OrgRank"] = stat1.apply(lambda x: round(x["Total Advancements of OrgRank"] / x["Total Transitions"] * 100,2) if x["Total Transitions"] > 0 else np.nan,axis=1)
stat1

,Year,Total Transitions,Total Advancements of OrgRank,Percentage Advancements of OrgRank
0,1946,4,2,50.00
1,1947,1,True,100.00
2,1948,3,3,100.00
3,1949,1,True,100.00
4,1950,8,5,62.50
...,...,...,...,...
71,2017,60,39,65.00
72,2018,44,27,61.36
73,2019,128,73,57.03
74,2020,49,22,44.90


In [166]:
stat1_filename = "orgrank_within_between.xlsx"
# stat1.to_excel(path_analysis + study0_path + stat2_filename,index=False)

### 2. OrgRank increase - percent by year, PISame = True

In [167]:
trans.columns

Index(['LeaderID', 'CareerString_1', 'CareerDateString_2022_1',
       'CareerStartYear_1', 'CareerStartMonth_1', 'CareerStartDate_1',
       'CareerSubstring_1', 'InstitutionType_1', 'PrimaryInstitution_1',
       'OrgName_1', 'Local_1', 'Position_1', 'IsElected_1', 'OrgRank_1',
       'PositionRank_1', 'CareerString_2', 'CareerDateString_2022_2',
       'CareerStartYear_2', 'CareerStartMonth_2', 'CareerStartDate_2',
       'CareerSubstring_2', 'InstitutionType_2', 'PrimaryInstitution_2',
       'OrgName_2', 'Local_2', 'Position_2', 'IsElected_2', 'OrgRank_2',
       'PositionRank_2', 'OrgAdvance', 'PositionAdvance', 'PISame', 'OrgSame',
       'OrgRankChange', 'PositionRankChange', 'Succession_Broad',
       'Succession_Narrow', 'OrgAdvanceYes', 'PositionAdvanceYes',
       'InstitutionCategory_1', 'InstitutionCategory_2',
       'InstitutionCategorySame', 'Succession_Five'],
      dtype='object')

In [168]:
stat2_columns = ["CareerStartYear_2","OrgAdvanceYes"]
stat2_condition = trans["PISame"]==True
stat2_groupby_columns = ["CareerStartYear_2"]
stat2_label_columns = ["Year","Total Transitions","Total Advancements of OrgRank"]

In [169]:
stat2 = trans.loc[stat2_condition,stat2_columns].groupby(stat2_groupby_columns,as_index=False)["OrgAdvanceYes"].agg({"Total Transitions":"count","Total Advancements of OrgRank":"sum"})
stat2.columns = stat2_label_columns
stat2["Percentage Advancements of OrgRank"] = stat2.apply(lambda x: round(x["Total Advancements of OrgRank"] / x["Total Transitions"] * 100,2) if x["Total Transitions"] > 0 else np.nan,axis=1)
stat2

,Year,Total Transitions,Total Advancements of OrgRank,Percentage Advancements of OrgRank
0,1946,2,0,0.00
1,1948,2,2,100.00
2,1950,4,1,25.00
3,1951,5,1,20.00
4,1952,3,2,66.67
...,...,...,...,...
66,2017,33,12,36.36
67,2018,21,4,19.05
68,2019,67,12,17.91
69,2020,34,7,20.59


In [170]:
# pivot SameInstitution - 

In [171]:
# stat2_filename = "orgrank_within.xlsx"
# stat2.to_excel(path_analysis + study0_path + stat2_filename,index=False)

### 3. OrgRank increase - percent by succesion period (broad)

In [172]:
trans.columns

Index(['LeaderID', 'CareerString_1', 'CareerDateString_2022_1',
       'CareerStartYear_1', 'CareerStartMonth_1', 'CareerStartDate_1',
       'CareerSubstring_1', 'InstitutionType_1', 'PrimaryInstitution_1',
       'OrgName_1', 'Local_1', 'Position_1', 'IsElected_1', 'OrgRank_1',
       'PositionRank_1', 'CareerString_2', 'CareerDateString_2022_2',
       'CareerStartYear_2', 'CareerStartMonth_2', 'CareerStartDate_2',
       'CareerSubstring_2', 'InstitutionType_2', 'PrimaryInstitution_2',
       'OrgName_2', 'Local_2', 'Position_2', 'IsElected_2', 'OrgRank_2',
       'PositionRank_2', 'OrgAdvance', 'PositionAdvance', 'PISame', 'OrgSame',
       'OrgRankChange', 'PositionRankChange', 'Succession_Broad',
       'Succession_Narrow', 'OrgAdvanceYes', 'PositionAdvanceYes',
       'InstitutionCategory_1', 'InstitutionCategory_2',
       'InstitutionCategorySame', 'Succession_Five'],
      dtype='object')

In [173]:
stat3_columns = ["Succession_Broad","OrgAdvanceYes"]
stat3_condition = trans["PISame"]==True
stat3_groupby_columns = ["Succession_Broad"]
stat3_label_columns = ["Succession (Broad)","Total Transitions","Total Advancements of OrgRank"]

In [174]:
stat3 = trans.loc[stat3_condition,stat3_columns].groupby(stat3_groupby_columns,as_index=False)["OrgAdvanceYes"].agg({"Total Transitions":"count","Total Advancements of OrgRank":"sum"})
stat3.columns = stat3_label_columns
stat3["Percentage Advancements of OrgRank"] = stat3.apply(lambda x: round(x["Total Advancements of OrgRank"] / x["Total Transitions"] * 100,2) if x["Total Transitions"] > 0 else np.nan,axis=1)
stat3

,Succession (Broad),Total Transitions,Total Advancements of OrgRank,Percentage Advancements of OrgRank
0,KIS to KJI,298,51,17.11
1,KJI to KJU,330,93,28.18


# Hypothesis 1a - Org Advancement, broad succession periods

In [175]:
select_columns = ["Succession_Broad","LeaderID"]
select_rows = trans["PISame"]==True
group_columns = ["Succession_Broad"]
trans.loc[select_rows,select_columns].groupby(group_columns,as_index=False).count()

,Succession_Broad,LeaderID
0,KIS to KJI,298
1,KJI to KJU,330


In [176]:
stat4_rows = trans["PISame"]==True
stat4_columns = ["Succession_Broad","OrgAdvanceYes"]
stat4_groupby = ["Succession_Broad"]
stat4_labels = ["Succession Period","Total Transitions","Total Advancements"]

In [177]:
stat4 = trans.loc[stat4_rows,stat4_columns].groupby(stat4_groupby,as_index=False)["OrgAdvanceYes"].agg({"Total Transitions":"count","Total Advancements":"sum"})
stat4

,Succession_Broad,Total Transitions,Total Advancements
0,KIS to KJI,298,51
1,KJI to KJU,330,93


In [178]:
stat4["Percentage Advancements of OrgRank"] = stat4.apply(lambda x: round(x["Total Advancements"] / x["Total Transitions"] * 100,2) if x["Total Transitions"] > 0 else np.nan,axis=1)
stat4

,Succession_Broad,Total Transitions,Total Advancements,Percentage Advancements of OrgRank
0,KIS to KJI,298,51,17.11
1,KJI to KJU,330,93,28.18


In [179]:
# 2-sample test of proportions
two_sample_ttest_proportions(51,93,298,330,.05)

p1:	 0.17114093959731544
p2:	 0.2818181818181818
p1-p2:	 -0.11067724222086636
z:	 -3.2945744504009844
pvalue:	 calculate


# Hypothesis 1b - Position Advancement, broad succession periods

In [180]:
select_columns = ["Succession_Broad","LeaderID"]
select_rows = (trans["PISame"] & trans["OrgSame"])==True
group_columns = ["Succession_Broad"]
trans.loc[select_rows,select_columns].groupby(group_columns,as_index=False).count()

,Succession_Broad,LeaderID
0,KIS to KJI,64
1,KJI to KJU,74


In [181]:
stat4b_rows = (trans["PISame"] & trans["OrgSame"])==True
stat4b_columns = ["Succession_Broad","PositionAdvanceYes"]
stat4b_groupby = ["Succession_Broad"]
stat4b_labels = ["Succession Period","Total Transitions","Total Advancements"]

In [182]:
stat4b = trans.loc[stat4b_rows,stat4b_columns].groupby(stat4b_groupby,as_index=False)["PositionAdvanceYes"].agg({"Total Transitions":"count","Total Advancements":"sum"})
stat4b

,Succession_Broad,Total Transitions,Total Advancements
0,KIS to KJI,64,20
1,KJI to KJU,74,22


In [183]:
stat4b["Percentage Advancements of Position Rank"] = stat4b.apply(lambda x: round(x["Total Advancements"] / x["Total Transitions"] * 100,2) if x["Total Transitions"] > 0 else np.nan,axis=1)
stat4b

,Succession_Broad,Total Transitions,Total Advancements,Percentage Advancements of Position Rank
0,KIS to KJI,64,20,31.25
1,KJI to KJU,74,22,29.73


In [184]:
# 2-sample test of proportions
two_sample_ttest_proportions(20,22,64,74,.05)

p1:	 0.3125
p2:	 0.2972972972972973
p1-p2:	 0.015202702702702686
z:	 0.19355568472355186
pvalue:	 calculate


# Hypothesis 1c - Org Advancement, broad succession, by Institution

In [185]:
select_rows = trans["PISame"]==True
select_columns = ["Succession_Broad","LeaderID"]
group_columns = ["Succession_Broad"]
trans.loc[select_rows,select_columns].groupby(group_columns,as_index=False).count()

,Succession_Broad,LeaderID
0,KIS to KJI,298
1,KJI to KJU,330


In [186]:
select_columns = ["InstitutionCategorySame","PISame","LeaderID"]
groupby_columns1 = ["InstitutionCategorySame"]
groupby_columns2 = ["PISame"]
groupby_columns3 = ["InstitutionCategorySame","PISame"]

In [187]:
trans[select_columns].groupby(groupby_columns3).count()

LeaderID
InstitutionCategorySame PISame          
False                   False       1952
True                    False        768
                        True        1502

In [188]:
ics = list(trans.InstitutionCategory_2.unique())
ics

['Government', 'Social', 'Party', 'Military']

In [189]:
for ic in ics:
    print("\n\n",ic,"\n\n")
    stat4c_rows = (trans["InstitutionCategory_2"]==ic) & (trans["PISame"]==True)
    stat4c_columns = ["Succession_Broad","OrgAdvanceYes"]
    stat4c_groupby = ["Succession_Broad"]
    stat4c_labels = ["Succession Period","Total Transitions","Total Advancements"]
    
    stat4c = trans.loc[stat4c_rows,stat4c_columns].groupby(stat4c_groupby,as_index=False)["OrgAdvanceYes"].agg({"Total Transitions":"count","Total Advancements":"sum"})    
    stat4c["Percentage Advancements of OrgRank"] = stat4c.apply(lambda x: round(x["Total Advancements"] / x["Total Transitions"] * 100,2) if x["Total Transitions"] > 0 else np.nan,axis=1)
   
    print(stat4c,"\n")
    
    p1 = stat4c.iloc[0,2]
    p2 = stat4c.iloc[1,2]
    n1 = stat4c.iloc[0,1]
    n2 = stat4c.iloc[1,1]
    
    two_sample_ttest_proportions(p1,p2,n1,n2,.05)



 Government 


  Succession_Broad  Total Transitions Total Advancements  \
0       KIS to KJI                116                 19   
1       KJI to KJU                190                 36   

   Percentage Advancements of OrgRank  
0                               16.38  
1                               18.95   

p1:	 0.16379310344827586
p2:	 0.18947368421052632
p1-p2:	 -0.025680580762250466
z:	 -0.5676140590926105
pvalue:	 calculate


 Social 


  Succession_Broad  Total Transitions Total Advancements  \
0       KIS to KJI                 27                  3   
1       KJI to KJU                 17                  2   

   Percentage Advancements of OrgRank  
0                               11.11  
1                               11.76   

p1:	 0.1111111111111111
p2:	 0.11764705882352941
p1-p2:	 -0.006535947712418305
z:	 -0.06651566609527824
pvalue:	 calculate


 Party 


  Succession_Broad  Total Transitions Total Advancements  \
0       KIS to KJI                143         

# Hypothesis 1d - Position Advancement, broad succession, by Institution

In [190]:
ics = list(trans.InstitutionCategory_2.unique())
for ic in ics:
    print("\n\n",ic,"\n\n")
    stat4c_rows = (trans["InstitutionCategory_2"]==ic) & (trans["PISame"]==True) & (trans["OrgSame"]==True)
    stat4c_columns = ["Succession_Broad","PositionAdvanceYes"]
    stat4c_groupby = ["Succession_Broad"]
    stat4c_labels = ["Succession Period","Total Transitions","Total Advancements"]
    
    stat4c = trans.loc[stat4c_rows,stat4c_columns].groupby(stat4c_groupby,as_index=False)["PositionAdvanceYes"].agg({"Total Transitions":"count","Total Advancements":"sum"})    
    stat4c["Percentage Advancements"] = stat4c.apply(lambda x: round(x["Total Advancements"] / x["Total Transitions"] * 100,2) if x["Total Transitions"] > 0 else np.nan,axis=1)
   
    print(stat4c,"\n")
    
    p1 = stat4c.iloc[0,2]
    p2 = stat4c.iloc[1,2]
    n1 = stat4c.iloc[0,1]
    n2 = stat4c.iloc[1,1]
    
    two_sample_ttest_proportions(p1,p2,n1,n2,.05)



 Government 


  Succession_Broad  Total Transitions Total Advancements  \
0       KIS to KJI                 21                  8   
1       KJI to KJU                 65                 16   

   Percentage Advancements  
0                    38.10  
1                    24.62   

p1:	 0.38095238095238093
p2:	 0.24615384615384617
p1-p2:	 0.13479853479853476
z:	 1.1972897301901602
pvalue:	 calculate


 Social 


  Succession_Broad  Total Transitions Total Advancements  \
0       KIS to KJI                  8                  5   
1       KJI to KJU                  5                  4   

   Percentage Advancements  
0                     62.5  
1                     80.0   

p1:	 0.625
p2:	 0.8
p1-p2:	 -0.17500000000000004
z:	 -0.6651023313078306
pvalue:	 calculate


 Party 


  Succession_Broad  Total Transitions Total Advancements  \
0       KIS to KJI                 34                  6   
1       KJI to KJU                  4                  2   

   Percentage Advancement

C:\Users\seoul\AppData\Local\Temp\ipykernel_13572\3713812013.py:5: RuntimeWarning: invalid value encountered in scalar divide
  p2 = x2/n2
C:\Users\seoul\AppData\Local\Temp\ipykernel_13572\3713812013.py:6: RuntimeWarning: divide by zero encountered in scalar divide
  z = (p1-p2) / math.sqrt(pstar*(1-pstar)*((1/n1) + (1/n2)))
C:\Users\seoul\AppData\Local\Temp\ipykernel_13572\3713812013.py:6: RuntimeWarning: invalid value encountered in scalar multiply
  z = (p1-p2) / math.sqrt(pstar*(1-pstar)*((1/n1) + (1/n2)))


# Hypothesis 2a - Org Advancement, narrow succession periods

In [90]:
select_columns = ["Succession_Narrow","LeaderID"]
select_rows = trans["PISame"]==True
group_columns = ["Succession_Narrow"]
trans.loc[select_rows,select_columns].groupby(group_columns,as_index=False).count()

,Succession_Narrow,LeaderID
0,KIS to KJI,145
1,KJI to KJU,164


In [91]:
stat4_rows = trans["PISame"]==True
stat4_columns = ["Succession_Narrow","OrgAdvanceYes"]
stat4_groupby = ["Succession_Narrow"]
stat4_labels = ["Succession Period","Total Transitions","Total Advancements"]

In [92]:
stat4 = trans.loc[stat4_rows,stat4_columns].groupby(stat4_groupby,as_index=False)["OrgAdvanceYes"].agg({"Total Transitions":"count","Total Advancements":"sum"})
stat4

,Succession_Narrow,Total Transitions,Total Advancements
0,KIS to KJI,145,25
1,KJI to KJU,164,63


In [93]:
stat4["Percentage Advancements of OrgRank"] = stat4.apply(lambda x: round(x["Total Advancements"] / x["Total Transitions"] * 100,2) if x["Total Transitions"] > 0 else np.nan,axis=1)
stat4

,Succession_Narrow,Total Transitions,Total Advancements,Percentage Advancements of OrgRank
0,KIS to KJI,145,25,17.24
1,KJI to KJU,164,63,38.41


In [94]:
# 2-sample test of proportions
two_sample_ttest_proportions(25,63,145,164,.05)

p1:	 0.1724137931034483
p2:	 0.38414634146341464
p1-p2:	 -0.21173254835996635
z:	 -4.11562045030938
pvalue:	 calculate


# Hypothesis 2b - Position Advancement, narrow succession periods

In [95]:
select_columns = ["Succession_Narrow","LeaderID"]
select_rows = (trans["PISame"] & trans["OrgSame"])==True
group_columns = ["Succession_Narrow"]
trans.loc[select_rows,select_columns].groupby(group_columns,as_index=False).count()

,Succession_Narrow,LeaderID
0,KIS to KJI,35
1,KJI to KJU,35


In [96]:
stat4b_rows = (trans["PISame"] & trans["OrgSame"])==True
stat4b_columns = ["Succession_Narrow","PositionAdvanceYes"]
stat4b_groupby = ["Succession_Narrow"]
stat4b_labels = ["Succession Period","Total Transitions","Total Advancements"]

In [97]:
stat4b = trans.loc[stat4b_rows,stat4b_columns].groupby(stat4b_groupby,as_index=False)["PositionAdvanceYes"].agg({"Total Transitions":"count","Total Advancements":"sum"})
stat4b

,Succession_Narrow,Total Transitions,Total Advancements
0,KIS to KJI,35,8
1,KJI to KJU,35,4


In [98]:
stat4b["Percentage Advancements of Position Rank"] = stat4b.apply(lambda x: round(x["Total Advancements"] / x["Total Transitions"] * 100,2) if x["Total Transitions"] > 0 else np.nan,axis=1)
stat4b

,Succession_Narrow,Total Transitions,Total Advancements,Percentage Advancements of Position Rank
0,KIS to KJI,35,8,22.86
1,KJI to KJU,35,4,11.43


In [99]:
# 2-sample test of proportions
two_sample_ttest_proportions(8,4,35,35,.05)

p1:	 0.22857142857142856
p2:	 0.11428571428571428
p1-p2:	 0.11428571428571428
z:	 1.2685406585123122
pvalue:	 calculate


# Hypothesis 2c - Org Advancement, narrow succession, by Institution

In [100]:
ics = list(trans.InstitutionCategory_2.unique())
for ic in ics:
    print("\n\n",ic,"\n\n")
    stat4c_rows = (trans["InstitutionCategory_2"]==ic) & (trans["PISame"]==True)
    stat4c_columns = ["Succession_Narrow","OrgAdvanceYes"]
    stat4c_groupby = ["Succession_Narrow"]
    stat4c_labels = ["Succession Period","Total Transitions","Total Advancements"]
    
    stat4c = trans.loc[stat4c_rows,stat4c_columns].groupby(stat4c_groupby,as_index=False)["OrgAdvanceYes"].agg({"Total Transitions":"count","Total Advancements":"sum"})    
    stat4c["Percentage Advancements of OrgRank"] = stat4c.apply(lambda x: round(x["Total Advancements"] / x["Total Transitions"] * 100,2) if x["Total Transitions"] > 0 else np.nan,axis=1)
   
    print(stat4c,"\n")
    
    p1 = stat4c.iloc[0,2]
    p2 = stat4c.iloc[1,2]
    n1 = stat4c.iloc[0,1]
    n2 = stat4c.iloc[1,1]
    
    two_sample_ttest_proportions(p1,p2,n1,n2,.05)



 Government 


  Succession_Narrow  Total Transitions Total Advancements  \
0        KIS to KJI                 65                 11   
1        KJI to KJU                 65                 12   

   Percentage Advancements of OrgRank  
0                               16.92  
1                               18.46   

p1:	 0.16923076923076924
p2:	 0.18461538461538463
p1-p2:	 -0.015384615384615385
z:	 -0.22983484344652486
pvalue:	 calculate


 Social 


  Succession_Narrow  Total Transitions Total Advancements  \
0        KIS to KJI                  7                  1   
1        KJI to KJU                  1              False   

   Percentage Advancements of OrgRank  
0                               14.29  
1                                0.00   

p1:	 0.14285714285714285
p2:	 0.0
p1-p2:	 0.14285714285714285
z:	 0.40406101782088427
pvalue:	 calculate


 Party 


  Succession_Narrow  Total Transitions Total Advancements  \
0        KIS to KJI                 66                 1

# Hypothesis 2d - Position Advancement, narrow succession, by Institution

In [101]:
ics = list(trans.InstitutionCategory_2.unique())
for ic in ics:
    print("\n\n",ic,"\n\n")
    stat4c_rows = (trans["InstitutionCategory_2"]==ic) & (trans["PISame"]==True) & (trans["OrgSame"]==True)
    stat4c_columns = ["Succession_Narrow","PositionAdvanceYes"]
    stat4c_groupby = ["Succession_Narrow"]
    stat4c_labels = ["Succession Period","Total Transitions","Total Advancements"]
    
    stat4c = trans.loc[stat4c_rows,stat4c_columns].groupby(stat4c_groupby,as_index=False)["PositionAdvanceYes"].agg({"Total Transitions":"count","Total Advancements":"sum"})    
    stat4c["Percentage Advancements"] = stat4c.apply(lambda x: round(x["Total Advancements"] / x["Total Transitions"] * 100,2) if x["Total Transitions"] > 0 else np.nan,axis=1)
   
    print(stat4c,"\n")
    
    p1 = stat4c.iloc[0,2]
    p2 = stat4c.iloc[1,2]
    n1 = stat4c.iloc[0,1]
    n2 = stat4c.iloc[1,1]
    
    two_sample_ttest_proportions(p1,p2,n1,n2,.05)



 Government 


  Succession_Narrow  Total Transitions Total Advancements  \
0        KIS to KJI                 16                  5   
1        KJI to KJU                 32                  2   

   Percentage Advancements  
0                    31.25  
1                     6.25   

p1:	 0.3125
p2:	 0.0625
p1-p2:	 0.25
z:	 2.3134209250170232
pvalue:	 calculate


 Social 


  Succession_Narrow  Total Transitions Total Advancements  \
0        KIS to KJI                  3                  0   
1        KJI to KJU                  0                  0   

   Percentage Advancements  
0                      0.0  
1                      NaN   

p1:	 0.0
p2:	 nan
p1-p2:	 nan
z:	 nan
pvalue:	 calculate


 Party 


  Succession_Narrow  Total Transitions Total Advancements  \
0        KIS to KJI                 15                  2   
1        KJI to KJU                  3                  2   

   Percentage Advancements  
0                    13.33  
1                    66.67   

p1:

C:\Users\seoul\AppData\Local\Temp\ipykernel_13572\3713812013.py:5: RuntimeWarning: invalid value encountered in scalar divide
  p2 = x2/n2
C:\Users\seoul\AppData\Local\Temp\ipykernel_13572\3713812013.py:6: RuntimeWarning: divide by zero encountered in scalar divide
  z = (p1-p2) / math.sqrt(pstar*(1-pstar)*((1/n1) + (1/n2)))
C:\Users\seoul\AppData\Local\Temp\ipykernel_13572\3713812013.py:6: RuntimeWarning: invalid value encountered in scalar multiply
  z = (p1-p2) / math.sqrt(pstar*(1-pstar)*((1/n1) + (1/n2)))
C:\Users\seoul\AppData\Local\Temp\ipykernel_13572\3713812013.py:5: RuntimeWarning: invalid value encountered in scalar divide
  p2 = x2/n2
C:\Users\seoul\AppData\Local\Temp\ipykernel_13572\3713812013.py:6: RuntimeWarning: divide by zero encountered in scalar divide
  z = (p1-p2) / math.sqrt(pstar*(1-pstar)*((1/n1) + (1/n2)))
C:\Users\seoul\AppData\Local\Temp\ipykernel_13572\3713812013.py:6: RuntimeWarning: invalid value encountered in scalar multiply
  z = (p1-p2) / math.sqrt(pst

# Hypothesis 3a - Org Advancement, succession period = Five

In [262]:
select_columns = ["Succession_Five","LeaderID"]
select_rows = trans["PISame"]==True
group_columns = ["Succession_Five"]
trans.loc[select_rows,select_columns].groupby(group_columns,as_index=False).count()

,Succession_Five,LeaderID
0,KIS to KJI,94
1,KJI to KJU,135


In [263]:
stat4_rows = trans["PISame"]==True
stat4_columns = ["Succession_Five","OrgAdvanceYes"]
stat4_groupby = ["Succession_Five"]
stat4_labels = ["Succession Period","Total Transitions","Total Advancements"]

In [264]:
stat4 = trans.loc[stat4_rows,stat4_columns].groupby(stat4_groupby,as_index=False)["OrgAdvanceYes"].agg({"Total Transitions":"count","Total Advancements":"sum"})
stat4

,Succession_Five,Total Transitions,Total Advancements
0,KIS to KJI,94,25
1,KJI to KJU,135,28


In [265]:
stat4["Percentage Advancements of OrgRank"] = stat4.apply(lambda x: round(x["Total Advancements"] / x["Total Transitions"] * 100,2) if x["Total Transitions"] > 0 else np.nan,axis=1)
stat4

,Succession_Five,Total Transitions,Total Advancements,Percentage Advancements of OrgRank
0,KIS to KJI,94,25,26.60
1,KJI to KJU,135,28,20.74


In [266]:
# 2-sample test of proportions
two_sample_ttest_proportions(25,28,94,135,.05)

p1:	 0.26595744680851063
p2:	 0.2074074074074074
p1-p2:	 0.05855003940110323
z:	 1.0334309766029885
pvalue:	 calculate


# Hypothesis 3b - Position Advancement, succession period = Five

In [267]:
select_columns = ["Succession_Five","LeaderID"]
select_rows = (trans["PISame"] & trans["OrgSame"])==True
group_columns = ["Succession_Five"]
trans.loc[select_rows,select_columns].groupby(group_columns,as_index=False).count()

,Succession_Five,LeaderID
0,KIS to KJI,15
1,KJI to KJU,26


In [268]:
stat4b_rows = (trans["PISame"] & trans["OrgSame"])==True
stat4b_columns = ["Succession_Five","PositionAdvanceYes"]
stat4b_groupby = ["Succession_Five"]
stat4b_labels = ["Succession Period","Total Transitions","Total Advancements"]

In [269]:
stat4b = trans.loc[stat4b_rows,stat4b_columns].groupby(stat4b_groupby,as_index=False)["PositionAdvanceYes"].agg({"Total Transitions":"count","Total Advancements":"sum"})
stat4b

,Succession_Five,Total Transitions,Total Advancements
0,KIS to KJI,15,7
1,KJI to KJU,26,11


In [270]:
stat4b["Percentage Advancements of Position Rank"] = stat4b.apply(lambda x: round(x["Total Advancements"] / x["Total Transitions"] * 100,2) if x["Total Transitions"] > 0 else np.nan,axis=1)
stat4b

,Succession_Five,Total Transitions,Total Advancements,Percentage Advancements of Position Rank
0,KIS to KJI,15,7,46.67
1,KJI to KJU,26,11,42.31


In [271]:
# 2-sample test of proportions
two_sample_ttest_proportions(7,11,15,26,.05)

p1:	 0.4666666666666667
p2:	 0.4230769230769231
p1-p2:	 0.0435897435897436
z:	 0.2708996074187647
pvalue:	 calculate


# Hypothesis 3c - Org Advancement, succession = Five, by Institution

In [272]:
ics = list(trans.InstitutionCategory_2.unique())
for ic in ics:
    print("\n\n",ic,"\n\n")
    stat4c_rows = (trans["InstitutionCategory_2"]==ic) & (trans["PISame"]==True)
    stat4c_columns = ["Succession_Five","OrgAdvanceYes"]
    stat4c_groupby = ["Succession_Five"]
    stat4c_labels = ["Succession Period","Total Transitions","Total Advancements"]
    
    stat4c = trans.loc[stat4c_rows,stat4c_columns].groupby(stat4c_groupby,as_index=False)["OrgAdvanceYes"].agg({"Total Transitions":"count","Total Advancements":"sum"})    
    stat4c["Percentage Advancements of OrgRank"] = stat4c.apply(lambda x: round(x["Total Advancements"] / x["Total Transitions"] * 100,2) if x["Total Transitions"] > 0 else np.nan,axis=1)
   
    print(stat4c,"\n")
    
    p1 = stat4c.iloc[0,2]
    p2 = stat4c.iloc[1,2]
    n1 = stat4c.iloc[0,1]
    n2 = stat4c.iloc[1,1]
    
    two_sample_ttest_proportions(p1,p2,n1,n2,.05)



 Government 


  Succession_Five  Total Transitions Total Advancements  \
0      KIS to KJI                 58                 17   
1      KJI to KJU                 62                 13   

   Percentage Advancements of OrgRank  
0                               29.31  
1                               20.97   

p1:	 0.29310344827586204
p2:	 0.20967741935483872
p1-p2:	 0.08342602892102333
z:	 1.0546786488216733
pvalue:	 calculate


 Social 


  Succession_Five  Total Transitions Total Advancements  \
0      KIS to KJI                 11                  1   
1      KJI to KJU                  5                  0   

   Percentage Advancements of OrgRank  
0                                9.09  
1                                0.00   

p1:	 0.09090909090909091
p2:	 0.0
p1-p2:	 0.09090909090909091
z:	 0.6963106238227914
pvalue:	 calculate


 Party 


  Succession_Five  Total Transitions Total Advancements  \
0      KIS to KJI                 16                  2   
1      KJI to KJ

# Hypothesis 3d - Position Advancement, Succession = Five, by Institution

In [273]:
ics = list(trans.InstitutionCategory_2.unique())
for ic in ics:
    print("\n\n",ic,"\n\n")
    stat4c_rows = (trans["InstitutionCategory_2"]==ic) & (trans["PISame"]==True) & (trans["OrgSame"]==True)
    stat4c_columns = ["Succession_Five","PositionAdvanceYes"]
    stat4c_groupby = ["Succession_Five"]
    stat4c_labels = ["Succession Period","Total Transitions","Total Advancements"]
    
    stat4c = trans.loc[stat4c_rows,stat4c_columns].groupby(stat4c_groupby,as_index=False)["PositionAdvanceYes"].agg({"Total Transitions":"count","Total Advancements":"sum"})    
    stat4c["Percentage Advancements"] = stat4c.apply(lambda x: round(x["Total Advancements"] / x["Total Transitions"] * 100,2) if x["Total Transitions"] > 0 else np.nan,axis=1)
   
    print(stat4c,"\n")
    
    p1 = stat4c.iloc[0,2]
    p2 = stat4c.iloc[1,2]
    n1 = stat4c.iloc[0,1]
    n2 = stat4c.iloc[1,1]
    
    two_sample_ttest_proportions(p1,p2,n1,n2,.05)



 Government 


  Succession_Five  Total Transitions Total Advancements  \
0      KIS to KJI                 11                  4   
1      KJI to KJU                 19                  5   

   Percentage Advancements  
0                    36.36  
1                    26.32   

p1:	 0.36363636363636365
p2:	 0.2631578947368421
p1-p2:	 0.10047846889952156
z:	 0.578729841692194
pvalue:	 calculate


 Social 


  Succession_Five  Total Transitions Total Advancements  \
0      KIS to KJI                  3                  2   
1      KJI to KJU                  1               True   

   Percentage Advancements  
0                    66.67  
1                   100.00   

p1:	 0.6666666666666666
p2:	 1.0
p1-p2:	 -0.33333333333333337
z:	 -0.6666666666666667
pvalue:	 calculate


 Party 


  Succession_Five  Total Transitions Total Advancements  \
0      KIS to KJI                  1               True   
1      KJI to KJU                  5                  5   

   Percentage Advanceme

C:\Users\seoul\AppData\Local\Temp\ipykernel_7912\3713812013.py:6: RuntimeWarning: invalid value encountered in scalar divide
  z = (p1-p2) / math.sqrt(pstar*(1-pstar)*((1/n1) + (1/n2)))
C:\Users\seoul\AppData\Local\Temp\ipykernel_7912\3713812013.py:4: RuntimeWarning: invalid value encountered in scalar divide
  p1 = x1/n1
C:\Users\seoul\AppData\Local\Temp\ipykernel_7912\3713812013.py:6: RuntimeWarning: divide by zero encountered in scalar divide
  z = (p1-p2) / math.sqrt(pstar*(1-pstar)*((1/n1) + (1/n2)))
C:\Users\seoul\AppData\Local\Temp\ipykernel_7912\3713812013.py:6: RuntimeWarning: invalid value encountered in scalar multiply
  z = (p1-p2) / math.sqrt(pstar*(1-pstar)*((1/n1) + (1/n2)))
